In [13]:
# import pre-processed data
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import applications, layers, losses, optimizers, Model
import keras_toolkit as kt
from sklearn.model_selection import train_test_split

pre_processed_data_path = '../20_intermediate_files/'

ImportError: cannot import name 'KaggleDatasets' from 'kaggledatasets' (/home/vscode/.venv/lib/python3.11/site-packages/kaggledatasets/__init__.py)

In [2]:
# load original image data
image_data_path = '../00_source_data/shopee-product-matching/train_images/'
train_csv = pd.read_csv('../00_source_data/shopee-product-matching/train.csv')

In [9]:
target_shape = (200, 200)
def preprocess_image(filename, target_shape=target_shape):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    img_str = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img_str, channels=3)
    img = tf.image.resize(img, target_shape)
    
    # Resnet-style preprocessing, see: https://git.io/JYo77
    mean = [103.939, 116.779, 123.68]
    img = img[..., ::-1]
    img -= mean

    return img


def build_triplets_dset(df, bsize=32, cache=True, shuffle=False):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """
    
    build_dataset = partial(
        kt.image.build_dataset,
        decode_fn=preprocess_image,
        bsize=bsize,
        cache=cache,
        shuffle=False
    )

    danchor = build_dataset(df.anchor)
    dpositive = build_dataset(df.positive)
    dnegative = build_dataset(df.negative)

    dset = tf.data.Dataset.zip((danchor, dpositive, dnegative))
    
    if shuffle:
        dset = dset.shuffle(shuffle)
    
    return dset



In [14]:
train = pd.read_csv('../00_source_data/shopee-product-matching/train.csv')

train = train.apply(lambda col: col.astype(str) if col.name == 'image' else col)
train_paths, val_paths = train_test_split(train, train_size=0.8, random_state=42)
train_paths.head()

,posting_id,image,image_phash,title,label_group
2014,train_3859949604,0f7a49e22f307a1405be8277bc6b34e3.jpg,dee4ed954989384a,Claris Kotak Makan 1250 ML SQ Foodsaver 2728,1.297383e+09
2238,train_2531150056,111cdfbb72d9eae0d81c339b74d6e21f.jpg,bfe4c0382f9ac287,Lampu LED Jagung T10 Senja/kota sein 10 mata 1...,2.339651e+09
485,train_70946930,03f0c92e2f79f3054547aab785bfb185.jpg,8c25c7eea55ba90c,Setelan koko anak cowok list mode | baju koko ...,3.974991e+09
2177,train_2100544538,10b9a25d3cb4d9e50089b957e6ce43ae.jpg,eb3384259ade1999,Tempered glass anti gores xiaomi mi band 4 9D ...,1.932824e+09
3534,train_4046796059,1b0a6d4e84c2de068ea130397da39b73.jpg,fc933c89cb1885e6,MUKENA DEWASA JUMBO BAHAN SATIN VELVET SUPER A...,2.153893e+09
